In [797]:
import os
import sys
import pandas
import numpy
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
import pydotplus
from IPython.display import Image
import subprocess

In [798]:
train = pandas.read_csv("train.csv",sep=",")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [799]:
le_sex = preprocessing.LabelEncoder()
train.Sex = le_sex.fit_transform(train.Sex)
length = train.Sex.shape[0]
sex = train.Sex.reshape(length,1)
survived = train.Survived.reshape(length,1).ravel()
pclass = train.Pclass.reshape(length,1)
train.Age = train.Age.copy()
meanAge = round(numpy.mean(train.Age[(numpy.isnan(train.Age)==False)]),0)
train.Age = train.Age.fillna(meanAge)
train.Age = train.Age.astype(int)
train.Age = pandas.qcut(train.Age, [0, .25, .5, .75, 1],labels=[train.Age.describe()['25%'],
                                                                train.Age.describe()['50%'],
                                                                train.Age.describe()['75%'],
                                                                train.Age.describe()['max']])
train.Age = train.Age.astype(int)
age = train.Age.reshape(length,1)
train.Embarked  = train.Embarked.fillna("S")
le_embar = preprocessing.LabelEncoder()
train.Embarked = le_embar.fit_transform(train.Embarked)
embarked = train.Embarked.reshape(length,1)
parch = train.Parch.reshape(length,1)
train.Fare = train.Fare.astype('int')
train.Fare = pandas.qcut(train.Fare, [0, .25, .5, .75, 1],labels=[train.Fare.describe()['25%'],
                                                                train.Fare.describe()['50%'],
                                                                train.Fare.describe()['75%'],
                                                                train.Fare.describe()['max']])
fare = train.Fare.reshape(length,1)
sibsp = train.SibSp.reshape(length,1)
names = train.Name.str.split(",").str.get(0)
le_names = preprocessing.LabelEncoder()
names = le_names.fit_transform(names)
names = names.reshape(length,1)

In [800]:
#data = pandas.DataFrame({'sex' : sex,'pclass' : pclass,'age' : age},index = list(range(train.shape[0])))
#pandas.concat([sex, pclass,age], axis=1)
data = pandas.DataFrame(sex)
data['age'] = age
data['pclass'] = pclass
data['embarked'] = embarked
data['parch'] = parch
data['fare'] = fare
#data['sibsp'] = sibsp
#data['names'] = names
data.columns = ['sex','age','pclass','embarked','parch','fare']

data.head()

,sex,age,pclass,embarked,parch,fare
0,1,22,3,2,0,7.0
1,0,80,1,0,0,512.0
2,0,30,3,2,0,7.0
3,0,35,1,2,0,512.0
4,1,35,3,2,0,14.0


In [801]:
names = train.Name.str.split(",").str.get(0)
families = names.value_counts()
nameList = families.index.to_series()
nameOccurences = pandas.DataFrame(nameList)
nameOccurences['numbers'] = families
nameOccurences.columns = ['Name','Occs']
train['JustNames'] = names
print(train.head())
print(nameOccurences.head())
nameOccurences['Survived'] = 0
for row in nameOccurences.Name:
    nameOccurences.ix[nameOccurences.Name==row,'Survived'] = sum(train.Survived[train.JustNames == row])
nameOccurences['SurvivingRation'] = nameOccurences.Survived/nameOccurences.Occs
print(nameOccurences.head(10))

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name  Sex  Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris    1   22      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    0   80      1      0   
2                             Heikkinen, Miss. Laina    0   30      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)    0   35      1      0   
4                           Allen, Mr. William Henry    1   35      0      0   

             Ticket   Fare Cabin  Embarked  JustNames  
0         A/5 21171    7.0   NaN         2     Braund  
1          PC 17599  512.0   C85         0    Cumings  
2  STON/O2. 3101282    7.0   NaN         2  Heikkinen  
3            113803  512.0  C123         2   Futrelle  
4            373450   

In [802]:
gnb = GaussianNB()
ypred = gnb.fit(data,survived).predict(sex)

In [803]:
print("Number of mislabeled points out of a total %d points : %d"
...       % (data.shape[0],(survived != ypred).sum()))

Number of mislabeled points out of a total 891 points : 190


In [804]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(data,survived)

In [805]:
dot_data = tree.export_graphviz(clf, 
                         feature_names=data.columns,  
                         class_names=True,  
                         filled=True, rounded=True)
tree.export_graphviz(clf, out_file='treepic.dot') 
#dot -Tpdf treepic.dot -o treepic.pdf

In [806]:
preds = clf.predict_proba(X = data)
pandas.crosstab(preds[:,0],columns = [data["sex"],data["pclass"],data["age"]])

sex        0                                   ...  1                        \
pclass     1               2             3     ...  1      2              3   
age       22  30  35  80  22 30  35 80  22  30 ... 35  80 22  30 35  80  22   
row_0                                          ...                            
0.000000  17  21  12  36  19  9  10  6   5  10 ...  5   1  7   1  0   0   5   
0.125000   0   0   0   0   0  8   0  0   0   0 ...  0   0  0   0  0   0   0   
0.166667   0   0   0   0   0  6   0  0   0   0 ...  0   0  0   0  0   0   0   
0.176471   0   0   0   0   0  0   0  0   0  17 ...  0   0  0   0  0   0   0   
0.200000   0   0   0   0   0  0   0  5   0   0 ...  0   0  0   0  0   0   0   
0.222222   0   0   0   0   0  0   0  9   0   0 ...  0   0  0   0  0   0   0   
0.250000   0   0   0   0   0  4   0  0  16   4 ...  0   0  0   0  0   0   0   
0.333333   3   3   0   0   0  0   0  0   6   3 ...  0   0  3   0  0   0   0   
0.400000   0   0   0   0   0  0   0  0   0   0 ...  0   0  0   0  0   0   0   
0.500000   0   0   0   2   0  0   0  0   6   2 ...  0   0  0   0  0   0   4   
0.555556   0   0   0   0   0  0   0  0   0   9 ...  0  27  0   0  0   0   0   
0.571429   0   0   0   0   0  0   0  0   0   0 ...  0   0  0   0  0   0   7   
0.600000   0   0   0   0   0  0   0  0   0   0 ...  0   5  0   0  0   0   5   
0.666667   0   0   0   0   0  0   0  0   3   3 ...  3   0  0   0  0   0   0   
0.700000   0   0   0   0   0  0   0  0   0   0 ...  0   0  0   0  0   0   0   
0.714286   0   0   0   0   0  0   0  0   7   0 ...  0   0  0   0  0   0   0   
0.750000   0   0   0   0   0  0   0  0   0   0 ...  0   4  0   0  8   0   0   
0.764706   0   0   0   0   0  0   0  0   0   0 ...  0  17  0   0  0   0   0   
0.777778   0   0   0   0   0  0   0  0   9   0 ...  0   0  0   0  0   0   0   
0.800000   0   0   0   0   0  0   0  0   0   5 ...  0   5  0   0  5   0   0   
0.833333   0   0   0   0   0  0   0  0   0   0 ...  0   0  0   0  0   0   0   
0.875000   0   0   0   0   0  0   0  0   0   0 ...  0   0  8   0  0  16   0   
0.878788   0   0   0   0   0  0   0  0   0   0 ...  0   0  0   0  0   0   0   
0.888889   0   0   0   0   0  0   0  0   0   0 ...  0   0  0   0  0   0  27   
0.894737   0   0   0   0   0  0   0  0   0   0 ...  0   0  0   0  0   0   0   
0.900000   0   0   0   0   0  0   0  0   0   0 ...  0   0  0   0  0   0   0   
0.909091   0   0   0   0   0  0   0  0   0   0 ...  0   0  0   0  0   0   0   
0.913043   0   0   0   0   0  0   0  0   0   0 ...  0   0  0   0  0   0   0   
0.941176   0   0   0   0   0  0   0  0   0   0 ...  0   0  0   0  0   0  34   
0.952381   0   0   0   0   0  0   0  0   0   0 ...  0   0  0  21  0   0   0   
1.000000   0   0   0   0   0  0   0  0   5  12 ...  1   7  4  17  4  14  22   

sex                   
pclass                
age       30  35  80  
row_0                 
0.000000   2   2   0  
0.125000   0   0   0  
0.166667   0   0   0  
0.176471   0   0   0  
0.200000   0   0   0  
0.222222   0   0   0  
0.250000   0   0   0  
0.333333   0   0   0  
0.400000   5   0   0  
0.500000   2   0   0  
0.555556   0   0   0  
0.571429   0   0   0  
0.600000   0   0   0  
0.666667   0   0   0  
0.700000   0   0   0  
0.714286   0   0   0  
0.750000   0  12   0  
0.764706   0   0   0  
0.777778   0   0   0  
0.800000   0   0   0  
0.833333   6   0   0  
0.875000   0   8   0  
0.878788  33   0   0  
0.888889  54   0   0  
0.894737  19   0   0  
0.900000  20   0   0  
0.909091   0   0  11  
0.913043   0   0  23  
0.941176   0   0   0  
0.952381   0   0   0  
1.000000  23   9  14  

[31 rows x 24 columns]

In [807]:
#Score the predictions
clf.score(X = data,y = train["Survived"])

0.86644219977553316

In [808]:
test = pandas.read_csv("test.csv",sep=",")
test.head()
test.shape

(418, 11)

In [809]:
meanFare = test.Fare[numpy.isnan(test.Fare) == False].mean()
test.Fare = test.Fare.fillna(meanFare)

In [810]:
le_sex = preprocessing.LabelEncoder()
test.Sex = le_sex.fit_transform(test.Sex)
length = test.Sex.shape[0]
sex = test.Sex.reshape(length,1)
pclass = test.Pclass.reshape(length,1)
test.Age = test.Age.copy()
meanAge = round(numpy.mean(test.Age[(numpy.isnan(test.Age)==False)]),0)
test.Age = test.Age.fillna(meanAge)
test.Age = test.Age.astype(int)
test.Age = pandas.qcut(test.Age, [0, .25, .5, .75, 1],labels=[test.Age.describe()['25%'],
                                                                test.Age.describe()['50%'],
                                                                test.Age.describe()['75%'],
                                                                test.Age.describe()['max']])
test.Age = test.Age.astype(int)
age = test.Age.reshape(length,1)
le_embar = preprocessing.LabelEncoder()
test.Embarked = le_embar.fit_transform(test.Embarked)
embarked = test.Embarked.reshape(length,1)
parch = test.Parch.reshape(length,1)
test.Fare = test.Fare.astype('int')
test.Fare = pandas.qcut(test.Fare, [0, .25, .5, .75, 1],labels=[test.Fare.describe()['25%'],
                                                                test.Fare.describe()['50%'],
                                                                test.Fare.describe()['75%'],
                                                                test.Fare.describe()['max']])
fare = test.Fare.reshape(length,1)
sibsp = test.SibSp.reshape(length,1)
names = test.Name.str.split(",").str.get(0)
le_names = preprocessing.LabelEncoder()
names = le_names.fit_transform(names)
names = names.reshape(length,1)

In [811]:
dataTest = pandas.DataFrame(sex)
dataTest['age'] = age
dataTest['pclass'] = pclass
dataTest['embarked'] = embarked
dataTest['parch'] = parch
dataTest['fare'] = fare
#dataTest['sibsp'] = sibsp
#dataTest['names'] = names
dataTest.columns = ['sex','age','pclass','embarked','parch','fare']

dataTest.T

,0,1,2,3,4,5,6,7,8,9,...,408,409,410,411,412,413,414,415,416,417
sex,1,0,1,1,0,1,0,1,0,1,...,0,0,0,0,0,1,0,1,1,1
age,35,76,76,30,23,23,30,30,23,23,...,30,23,30,76,30,30,76,76,30,30
pclass,3,3,2,3,3,3,3,2,3,3,...,3,3,3,1,3,3,1,3,3,3
embarked,1,2,1,2,2,2,1,2,0,2,...,1,2,1,1,2,2,0,2,2,0
parch,0,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
fare,7,7,14,14,14,14,7,31,7,31,...,7,14,7,512,7,14,512,7,14,31


In [812]:
names = test.Name.str.split(",").str.get(0)
test['JustNames'] = names
test['ProbaSurvived'] = -1.0
for row in nameOccurences.Name:
    if row in test.JustNames.values:
        test.ix[test.JustNames == row,'ProbaSurvived'] = (nameOccurences.SurvivingRation[nameOccurences.Name == row].values)
print(test.head())
cpt = 0
for proba in test.ProbaSurvived:
    if proba >= 0.5:
        cpt = cpt+1
print(cpt)

   PassengerId  Pclass                                          Name  Sex  \
0          892       3                              Kelly, Mr. James    1   
1          893       3              Wilkes, Mrs. James (Ellen Needs)    0   
2          894       2                     Myles, Mr. Thomas Francis    1   
3          895       3                              Wirz, Mr. Albert    1   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)    0   

   Age  SibSp  Parch   Ticket  Fare Cabin  Embarked JustNames  ProbaSurvived  
0   35      0      0   330911   7.0   NaN         1     Kelly           0.75  
1   76      1      0   363272   7.0   NaN         2    Wilkes          -1.00  
2   76      0      0   240276  14.0   NaN         1     Myles          -1.00  
3   30      0      0   315154  14.0   NaN         2      Wirz          -1.00  
4   23      1      1  3101298  14.0   NaN         2  Hirvonen           1.00  
97


In [813]:
# Make test set predictions
test_preds = clf.predict(X=dataTest)

# Create a submission for Kaggle
submission = pandas.DataFrame({"PassengerId":test["PassengerId"],
                           "Survived":test_preds})

for ids in test.PassengerId:
    if(test.ProbaSurvived[test.PassengerId == ids].values >= 0.75) & (submission.Survived[submission.PassengerId == ids].values == 0):
        submission.Survived[submission.PassengerId == ids] = 1

print(submission.Survived.head(10))

0    0
1    0
2    0
3    0
4    1
5    0
6    1
7    0
8    1
9    0
Name: Survived, dtype: int64


In [814]:
tt = pandas.concat([test,submission],axis=1)

tt.to_csv("train_submission.csv", 
                  index=False) 

In [815]:
# Save submission to CSV
submission.to_csv("trees_submission.csv", 
                  index=False)  